In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from env import username, password, hostname

In [ ]:
# url = f'mysql+pymysql://{username}:{password}@{hostname}/curriculum_logs'
# query = """select * from logs join cohorts on logs.cohort_id = cohorts.id;"""
# df = pd.read_sql(query, url)
# df.drop("id", axis=1, inplace=True)
# df.to_csv("curriculum_logs.csv", index=False)
# df.head()

In [ ]:
df = pd.read_csv("curriculum_logs.csv")
df.head(2)

In [ ]:
df["program_name"] = np.where(df["program_id"] == 3, "data science", "web dev")
df.head(2)

In [ ]:
df["date_time"] = pd.to_datetime(df['date'] + " " + df['time'])
df.head(2)

In [ ]:
df = df.set_index(df["date_time"]).drop(["date", "time", "date_time"], axis=1)
df.head(2)

In [45]:
from wrangle import Wrangle
df = Wrangle().prep_data()
df.head()

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_ name
date_time,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,web dev


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


In [40]:
df.groupby(by=["program_name", "name", "path"]).count()["user_id"][1:].sort_values(ascending=False)

program_name  name       path                                                            
web dev       Staff      /                                                                   6340
data science  Darden     /                                                                   2980
web dev       Voyageurs  /                                                                   2101
              Fortuna    /                                                                   2038
              Jupiter    toc                                                                 1866
                                                                                             ... 
              Quincy     content/php_iv/classes-and-objects-ii                                  1
                         content/php_iv/classes-and-objects-ii/gitbook/images/favicon.ico       1
                         content/php_iv/php-with-mysql                                          1
                         con

2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
- These are the top 10 cohorts who accessed the curriculum the most
### DONE

In [ ]:
df[["name", "path"]][df["name"] != "Staff"].groupby(by=["name"], sort=False).count().nlargest(n=10, columns="path", keep="first")

3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
- These students were active students and not staff. These are the 10 students whom accessed the curriculum the fewest
### DONE

In [ ]:
was_student = (df.index < df["end_date"]) & (df.index > df["start_date"])
not_staff = (df["name"] != "Staff")

df[["user_id", "path"]][was_student & not_staff].groupby(by=["user_id"], sort=False).count().nsmallest(n=10, columns="path", keep="first")

4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?


In [41]:
df.head(2)

,path,user_id,cohort_id,ip,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,program_name
date_time,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,web dev


5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?


6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
- These are the topics by program that got referenced after a student graduated.
### DONE

In [ ]:
df["path"][(df.index > df["end_date"]) & (df["program_name"] == "web dev") & (df["name"] != "Staff")]

In [ ]:
df["path"][(df.index > df["end_date"]) & (df["program_name"] == "data science") & (df["name"] != "Staff")]

7. Which lessons are least accessed?
- These are the top 50 least accessed topics, but this can be adjusted based on needs
### DONE

In [ ]:
df1["path"].value_counts(ascending=True).nsmallest(n=50, keep="first").to_dict()